#  SQL Queries 02

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
%load_ext sql

In [4]:
%sql sqlite://

'Connected: @None'

## Create toy data set

In [5]:
from faker import Faker

In [6]:
fake = Faker()

In [7]:
names = list(set([fake.name() for i in range(100)]))

In [8]:
np.random.seed(123)

n1 = 30
bios821 = pd.DataFrame(dict(
    name=np.random.choice(names, n1, replace=False), 
    grade=np.random.randint(50, 101, n1)))

n2 = 30
bios823 = pd.DataFrame(dict(
    name=np.random.choice(names, n2, replace=False), 
    grade=np.random.randint(50, 101, n2)))

In [9]:
%%sql

DROP TABLE IF EXISTS bios821;
DROP TABLE IF EXISTs bios823

 * sqlite://
Done.
Done.


[]

In [10]:
%sql -p bios821

 * sqlite://
(sqlite3.OperationalError) near "-": syntax error
[SQL: -p bios821]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [11]:
%sql -p bios823

 * sqlite://
(sqlite3.OperationalError) near "-": syntax error
[SQL: -p bios823]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [12]:
%sql SELECT * FROM bios821 LIMIT 3

 * sqlite://
(sqlite3.OperationalError) no such table: bios821
[SQL: SELECT * FROM bios821 LIMIT 3]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [13]:
%sql SELECT * FROM bios823 LIMIT 3

 * sqlite://
(sqlite3.OperationalError) no such table: bios823
[SQL: SELECT * FROM bios823 LIMIT 3]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


## Subqueries

There are 3 ways to use a sub-query:

- with `in`
- with `exists`
- with a comparison operator

For simplicity, we will treat the name as a unique identifier for each row.

**1**. What students take both bios821 and bios823?

#### Using set operations.

In [14]:
%%sql

SELECT name FROM bios821
INTERSECT
SELECT name FROM bios823

 * sqlite://
(sqlite3.OperationalError) no such table: bios823
[SQL: SELECT name FROM bios821
INTERSECT
SELECT name FROM bios823]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


#### Using EQUIJOIN.

In [15]:
%%sql

SELECT DISTINCT bios821.name 
FROM bios821, bios823
WHERE bios821.name = bios823.name

 * sqlite://
(sqlite3.OperationalError) no such table: bios821
[SQL: SELECT DISTINCT bios821.name 
FROM bios821, bios823
WHERE bios821.name = bios823.name]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


#### Using inner join

In [14]:
%%sql

SELECT DISTINCT bios821.name
FROM bios821
INNER JOIN bios823
ON bios821.name = bios823.name

 * sqlite://
(sqlite3.OperationalError) no such table: bios821
[SQL: SELECT DISTINCT bios821.name
FROM bios821
INNER JOIN bios823
ON bios821.name = bios823.name]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


#### Using sub-query.

In a standard subquery, the subquery runs once then is available for use by the outer query.

In [16]:
%%sql

SELECT DISTINCT name FROM bios821
WHERE name IN (
    SELECT name FROM bios823
)

 * sqlite://
(sqlite3.OperationalError) no such table: bios821
[SQL: SELECT DISTINCT name FROM bios821
WHERE name IN (
    SELECT name FROM bios823
)]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


#### Using correlated sub-query.

A correlated subquery is evaluated once for each row processed by the outer query.

In [17]:
%%sql

SELECT DISTINCT name FROM bios821
WHERE EXISTS (
    SELECT * FROM bios823 
    WHERE bios821.name = bios823.name -- bios821.name comes from the outer query
)

 * sqlite://
(sqlite3.OperationalError) no such table: bios821
[SQL: SELECT DISTINCT name FROM bios821
WHERE EXISTS (
    SELECT * FROM bios823 
    WHERE bios821.name = bios823.name -- bios821.name comes from the outer query
)]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


**2**. Which students taking bios823 have grades higher than the average grade of bios821? 

- Do this for all students
- Do this just for the subset of students taking both classes

In [18]:
%%sql

SELECT DISTINCT name, grade FROM bios823
WHERE grade > (
    SELECT AVG(grade) FROM bios821 
)

 * sqlite://
(sqlite3.OperationalError) no such table: bios823
[SQL: SELECT DISTINCT name, grade FROM bios823
WHERE grade > (
    SELECT AVG(grade) FROM bios821 
)]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


#### Using a view.

This is used to find the subset of students taking both classes.

In [19]:
%%sql

DROP VIEW IF EXISTS view_common;
CREATE VIEW view_common AS 
    SELECT DISTINCT
        bios821.name, 
        bios821.grade as grade_821, 
        bios823.grade as grade_823 
    FROM 
        bios821, bios823
    WHERE 
        bios821.name = bios823.name
  

 * sqlite://
Done.
Done.


[]

In [20]:
%%sql

SELECT name, grade_823
FROM view_common
WHERE grade_823 > (
    SELECT AVG(grade_821) 
    FROM view_common
)

 * sqlite://
(sqlite3.OperationalError) no such table: main.bios821
[SQL: SELECT name, grade_823
FROM view_common
WHERE grade_823 > (
    SELECT AVG(grade_821) 
    FROM view_common
)]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


#### Using `pandas`.

In [21]:
df = bios821.merge(bios823, on='name', suffixes=['_821', '_823'])

In [22]:
df

,name,grade_821,grade_823
0,Arthur Lopez MD,75,73
1,Amber Lynch,50,67
2,Steven Berry,51,86
3,Joel Rivera,88,64
4,Ashley Adams,51,74
5,Brenda Harrison,56,73
6,Corey Garner,63,91
7,Lindsey Peterson,78,98


In [23]:
df[df.grade_823 > df.grade_821.mean()]

,name,grade_821,grade_823
0,Arthur Lopez MD,75,73
1,Amber Lynch,50,67
2,Steven Berry,51,86
4,Ashley Adams,51,74
5,Brenda Harrison,56,73
6,Corey Garner,63,91
7,Lindsey Peterson,78,98


## Common table expressions (CTE)

CTEs are temporary tables created for a specific query.

### Basic syntax for CTE

In [15]:
%%sql

WITH
    t1(name) AS (SELECT 'Bob'),
    t2(age) AS (SELECT 23)
SELECT * from t1, t2

 * sqlite://
Done.


name,age
Bob,23


#### Using CTEs to solve previous problem

In [ ]:
%%sql

WITH common AS (
    SELECT DISTINCT 
        bios821.name, 
        bios821.grade as grade_821, 
        bios823.grade as grade_823 
    FROM 
        bios821,
        bios823
    WHERE 
        bios821.name = bios823.name
)
SELECT name, grade_823
FROM common
WHERE grade_823 > (
    SELECT AVG(grade_821) 
    FROM common
)

### Review of recursive functions

Write a recursive function that returns the first $n$ Fibonacci numbers. The Fibonacci sequence is $1,1,2,3,5,8,\ldots$.

Non-recursive version

In [16]:
def fib_nr(n):
    a, b = 1, 1
    for i in range(n):
        a, b = b, a+b
    return a

In [17]:
fib_nr(10)

89

Now write the recursive version as `fib_r`

In [18]:
def fib_r(n):
    if n == 0: return 1
    elif n == 1: return 1
    else: return (fib_r(n-1) + fib_r(n-2))


In [19]:
fib_r(10)

89

### Recursive CTEs

Image source: https://www.sqlservertutorial.net/wp-content/uploads/SQL-Server-Recursive-CTE-execution-flow.png
![img](https://www.sqlservertutorial.net/wp-content/uploads/SQL-Server-Recursive-CTE-execution-flow.png)

#### As a generator

Note `UNION ALL` is like `UNION` but does not remove duplicates.

In [20]:
%%sql

WITH RECURSIVE count(n) AS (
    SELECT 1
        UNION ALL
    SELECT n+1 FROM count WHERE n < 5
)
SELECT * FROM count

 * sqlite://
Done.


n
1
2
3
4
5


#### Generating dates

In [21]:
%%sql

WITH RECURSIVE make_dates(d) AS (
    SELECT '2021-01-15'
        UNION ALL
    SELECT DATE(d, '+1 MONTHS') FROM make_dates WHERE d < '2021-12'
)
SELECT d AS "Homework due" FROM make_dates

 * sqlite://
Done.


Homework due
2021-01-15
2021-02-15
2021-03-15
2021-04-15
2021-05-15
2021-06-15
2021-07-15
2021-08-15
2021-09-15
2021-10-15


#### CTEs to do programming in SQL(!)

In [22]:
%%sql

WITH RECURSIVE fact(n, f) AS (
    SELECT 1, 1
        UNION ALL
    SELECT n+1, (n+1) * f 
    FROM fact 
    WHERe n < 5
)
SELECT * FROM fact
LIMIT 5

 * sqlite://
Done.


n,f
1,1
2,2
3,6
4,24
5,120


In [23]:
%%sql

WITH RECURSIVE fib(n, f1, f2) AS (
    SELECT 1, 1, 1
        UNION ALL
    SELECT n+1, f2, f1+f2
    FROM fib 
    WHERe n < 10
)
SELECT * FROM fib
LIMIT 10

 * sqlite://
Done.


n,f1,f2
1,1,1
2,1,2
3,2,3
4,3,5
5,5,8
6,8,13
7,13,21
8,21,34
9,34,55
10,55,89


#### A common use of CTEs is to work with naturally recursive structures (trees or graphs)

Suppose we have a cell subset taxonomy with parent-child relations. We want to find all `descendants` of a particular cell type. This is hard to do without recursive CTEs in SQL. 

In [24]:
%%sql sqlite:///

DROP TABLE IF EXISTS cell;

CREATE TABLE cell(
    cell_id integer PRIMARY KEY,
    name VARCHAR(30),
    parent_id integer
);

INSERT INTO cell (
   cell_id,
   name,
   parent_id
)
VALUES
   (1, 'WBC', NULL),
   (2, 'Lymphocyte', 1),
   (3, 'T Cell', 2),
   (4, 'B Cell', 2),
   (5, 'NK Cell', 2),
   (6, 'T helper cell', 3),
   (7, 'T cytotoxic cell', 3),
   (8, 'T regulatory cell', 3),
   (9, 'Naive B cell', 4),
   (10, 'Memory B cell', 4),
   (11, 'Plasma cell', 4),
   (12, 'Granulocyte', 1),
   (13, 'Basophil', 12),
   (14, 'Eosinophil', 12),
   (15, 'Neutrophil', 12)

Done.
Done.
15 rows affected.


[]

In [25]:
%%sql

SELECT * FROM cell LIMIT 10

   sqlite://
 * sqlite:///
Done.


cell_id,name,parent_id
1,WBC,None
2,Lymphocyte,1
3,T Cell,2
4,B Cell,2
5,NK Cell,2
6,T helper cell,3
7,T cytotoxic cell,3
8,T regulatory cell,3
9,Naive B cell,4
10,Memory B cell,4


In [26]:
%%sql

SELECT cell_id, name, parent_id
FROM cell
WHERE name='T Cell'

   sqlite://
 * sqlite:///
Done.


cell_id,name,parent_id
3,T Cell,2


In [27]:
target = 'Lymphocyte'

In [28]:
%%sql

WITH RECURSIVE lineage AS (
    SELECT 
        cell_id, 
        name, 
        parent_id
    FROM 
        cell
    WHERE 
        name=:target -- This is how we pass Python variables to SQL in `sql` magic
UNION ALL
    SELECT
        c.cell_id, 
        c.name, 
        c.parent_id
    FROM 
        cell c
    INNER JOIN 
        lineage l 
    ON 
        l.cell_id = c.parent_id
)
SELECT * FROM lineage 
LIMIT 10

   sqlite://
 * sqlite:///
Done.


cell_id,name,parent_id
2,Lymphocyte,1
4,B Cell,2
5,NK Cell,2
3,T Cell,2
10,Memory B cell,4
9,Naive B cell,4
11,Plasma cell,4
7,T cytotoxic cell,3
6,T helper cell,3
8,T regulatory cell,3


## Window Functions

- [Official Refs](https://www.sqlite.org/windowfunctions.html)
![img](https://www.sqlitetutorial.net/wp-content/uploads/2018/11/SQLite-Window-Functions-1.png)

In [29]:
np.random.seed(23)
n = 10
df = pd.DataFrame(
    dict(person=np.random.choice(['A', 'B', 'C', 'D'], n,),
                time=np.random.randint(0, 10, n), 
                bsl=np.random.randint(50, 400, n)))

In [30]:
df.sort_values(['person', 'time'])

,person,time,bsl
8,A,0,115
5,A,2,237
2,A,3,129
7,B,5,86
3,B,6,396
4,C,1,107
1,C,9,347
6,D,5,89
9,D,5,221
0,D,7,98


In [31]:
%sql DROP TABLE IF EXISTS df

   sqlite://
 * sqlite:///
Done.


[]

Magic shortcut to creating a database table from `pandas` DataFrame.

In [35]:
%sql -p df

   sqlite://
 * sqlite:///
(sqlite3.OperationalError) near "-": syntax error
[SQL: -p df]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


### Over  creates widows

I've given `pandas` equivalents where possible, but sometimes they are rather unnatural.

In [33]:
%%sql

SELECT person, time, bsl, 
row_number() OVER () as row_num
FROM df;

   sqlite://
 * sqlite:///
(sqlite3.OperationalError) no such table: df
[SQL: SELECT person, time, bsl, row_number() 
OVER () as row_num
FROM df;]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [36]:
df.assign(row_num =  df.person.expanding(1).count().astype('int'))

,person,time,bsl,row_num
0,D,7,98,1
1,C,9,347,2
2,A,3,129,3
3,B,6,396,4
4,C,1,107,5
5,A,2,237,6
6,D,5,89,7
7,B,5,86,8
8,A,0,115,9
9,D,5,221,10


### Lag and Lead

In [37]:
%%sql

SELECT person, time, bsl, 
lag(bsl, 1) OVER () as lag1, -- looks one row "behind"
lead(bsl, 2) OVER () as lead2 -- loosk two rows "ahead"
FROM df;

   sqlite://
 * sqlite:///
(sqlite3.OperationalError) no such table: df
[SQL: SELECT person, time, bsl, 
lag(bsl, 1) OVER () as lag1, -- looks one row "behind"
lead(bsl, 2) OVER () as lead2 -- loosk two rows "ahead"
FROM df;]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [38]:
df_ = df.copy()
df_['lag1'] = df_.bsl.shift(1)
df_['lead2'] = df_.bsl.shift(-2)
df_

,person,time,bsl,lag1,lead2
0,D,7,98,NaN,129.0
1,C,9,347,98.0,396.0
2,A,3,129,347.0,107.0
3,B,6,396,129.0,237.0
4,C,1,107,396.0,89.0
5,A,2,237,107.0,86.0
6,D,5,89,237.0,115.0
7,B,5,86,89.0,221.0
8,A,0,115,86.0,NaN
9,D,5,221,115.0,NaN


### Order by

In [39]:
%%sql

SELECT person, time, bsl, row_number() 
OVER (ORDER BY person, time)
FROM df;

   sqlite://
 * sqlite:///
(sqlite3.OperationalError) no such table: df
[SQL: SELECT person, time, bsl, row_number() 
OVER (ORDER BY person, time)
FROM df;]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [40]:
df_ = df.copy()
df_ = df_.sort_values(['person', 'time'])
df_.assign(row_num =  df_.person.expanding(1).count().astype('int'))

,person,time,bsl,row_num
8,A,0,115,1
5,A,2,237,2
2,A,3,129,3
7,B,5,86,4
3,B,6,396,5
4,C,1,107,6
1,C,9,347,7
6,D,5,89,8
9,D,5,221,9
0,D,7,98,10


### Partition by

In [41]:
%%sql

SELECT person, time, bsl, row_number() 
OVER (PARTITION BY person ORDER BY time) as row_number
FROM df;

   sqlite://
 * sqlite:///
(sqlite3.OperationalError) no such table: df
[SQL: SELECT person, time, bsl, row_number() 
OVER (PARTITION BY person ORDER BY time) as row_number
FROM df;]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [42]:
df_ = df.copy()
df_ = df_.sort_values(['person', 'time'])
df_['row_number'] = df_.groupby(['person']).cumcount()+1
df_

,person,time,bsl,row_number
8,A,0,115,1
5,A,2,237,2
2,A,3,129,3
7,B,5,86,1
3,B,6,396,2
4,C,1,107,1
1,C,9,347,2
6,D,5,89,1
9,D,5,221,2
0,D,7,98,3


In [43]:
%%sql

SELECT person, time, bsl, group_concat(bsl, ', ')
OVER (PARTITION BY person ORDER BY time) as window
FROM df;

   sqlite://
 * sqlite:///
(sqlite3.OperationalError) no such table: df
[SQL: SELECT person, time, bsl, group_concat(bsl, ', ')
OVER (PARTITION BY person ORDER BY time) as window
FROM df;]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [44]:
df_['window'] = (
    df_.groupby('person').
    apply(lambda x: pd.Series([x.bsl.iloc[:(i+1)].values 
                               for i in pd.Series(np.arange(len(x)))]))
).values
df_['window'] = df_.window.apply(lambda x: ','.join(map(str, x)))
df_

,person,time,bsl,row_number,window
8,A,0,115,1,115
5,A,2,237,2,"115,237"
2,A,3,129,3,"115,237,129"
7,B,5,86,1,86
3,B,6,396,2,"86,396"
4,C,1,107,1,107
1,C,9,347,2,"107,347"
6,D,5,89,1,89
9,D,5,221,2,"89,221"
0,D,7,98,3,"89,221,98"


### Specifying rows in window

In [45]:
%%sql

SELECT person, time, bsl, group_concat(bsl, ', ')
OVER (
    PARTITION BY person
    ORDER BY time
    ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING
     ) as window
FROM df;

   sqlite://
 * sqlite:///
(sqlite3.OperationalError) no such table: df
[SQL: SELECT person, time, bsl, group_concat(bsl, ', ')
OVER (
    PARTITION BY person
    ORDER BY time
    ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING
     ) as window
FROM df;]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [46]:
df_['window'] = (
    df_.groupby('person').
    apply(lambda x: pd.Series([x.bsl.iloc[(i-len(x)-1):(i+2)].values 
                               for i in pd.Series(np.arange(len(x)))]))
).values
df_['window'] = df_.window.apply(lambda x: ','.join(map(str, x)))
df_

,person,time,bsl,row_number,window
8,A,0,115,1,"115,237"
5,A,2,237,2,"115,237,129"
2,A,3,129,3,"237,129"
7,B,5,86,1,"86,396"
3,B,6,396,2,"86,396"
4,C,1,107,1,"107,347"
1,C,9,347,2,"107,347"
6,D,5,89,1,"89,221"
9,D,5,221,2,"89,221,98"
0,D,7,98,3,"221,98"


### Using window functions

In [47]:
%%sql

SELECT person, time, bsl, 
       row_number() OVER win    AS row_number,
       rank() OVER win          AS rank,
       dense_rank() OVER win    AS dense_rank,
       percent_rank() OVER win  AS percent_rank,
       cume_dist() OVER win     AS cume_dist
FROM df
WINDOW win AS (ORDER BY person);

   sqlite://
 * sqlite:///
(sqlite3.OperationalError) no such table: df
[SQL: SELECT person, time, bsl, 
       row_number() OVER win    AS row_number,
       rank() OVER win          AS rank,
       dense_rank() OVER win    AS dense_rank,
       percent_rank() OVER win  AS percent_rank,
       cume_dist() OVER win     AS cume_dist
FROM df
WINDOW win AS (ORDER BY person);]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [48]:
df_ = df.copy()
df_ = df_.sort_values(['person'])
df_['row_num'] = df_['person'].expanding(1).count().astype('int')
df_['rank'] = df_['person'].rank(method='min').astype('int')
df_['dense_rank'] = df_['person'].rank(method='dense').astype('int')
df_['percent_rank'] = (df_.person.rank(method='min') - 1) / (df_.person.count()-1)
df_['cume_dist'] = df_['person'].rank(method='max', pct=True)
df_

,person,time,bsl,row_num,rank,dense_rank,percent_rank,cume_dist
2,A,3,129,1,1,1,0.000000,0.3
5,A,2,237,2,1,1,0.000000,0.3
8,A,0,115,3,1,1,0.000000,0.3
3,B,6,396,4,4,2,0.333333,0.5
7,B,5,86,5,4,2,0.333333,0.5
1,C,9,347,6,6,3,0.555556,0.7
4,C,1,107,7,6,3,0.555556,0.7
0,D,7,98,8,8,4,0.777778,1.0
6,D,5,89,9,8,4,0.777778,1.0
9,D,5,221,10,8,4,0.777778,1.0


### The NTILE window function

In [49]:
scores = pd.DataFrame(dict(scores=np.random.randint(0, 10, 10)))
scores

,scores
0,6
1,7
2,3
3,6
4,9
5,2
6,3
7,0
8,8
9,6


In [50]:
%sql -p scores

   sqlite://
 * sqlite:///
(sqlite3.OperationalError) near "-": syntax error
[SQL: -p scores]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [51]:
%%sql

SELECT scores, NTILE(4)
OVER (
    -- PARTITION BY scores
    ORDER BY scores 
) AS quartile
FROM scores

   sqlite://
 * sqlite:///
(sqlite3.OperationalError) no such table: scores
[SQL: SELECT scores, NTILE(4)
OVER (
    -- PARTITION BY scores
    ORDER BY scores 
) AS quartile
FROM scores]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


Note: See [this](https://stackoverflow.com/questions/20726493/python-pandas-qcut-behavior-with-of-observations-not-divisible-by-of-bins) for an explanation of the difference between NTILE and `qcut`

In [52]:
quartiles = pd.qcut(scores.scores, 4, labels=[1,2,3,4])
scores['quantile'] = quartiles
scores.sort_values('scores')

,scores,quantile
7,0,1
5,2,1
2,3,1
6,3,1
0,6,2
3,6,2
9,6,2
1,7,4
8,8,4
4,9,4


### Using aggregate functions

The use of `LAST_VALUE` is over the current window, which by default runs to the *current* row.

In [53]:
%%sql

SELECT person, time, bsl,
       SUM(bsl) OVER win    AS bsl_sum,
       AVG(bsl) OVER win    AS bsl_avg,
       MIN(bsl) OVER win    AS bsl_min,
       MAX(bsl) over win as bsl_max,
       FIRST_VALUE(bsl) OVER win as bsl_start,
       LAST_VALUE(bsl) OVER win as bsl_end 
FROM df
WINDOW win AS (PARTITION BY person ORDER BY time) 

   sqlite://
 * sqlite:///
(sqlite3.OperationalError) no such table: df
[SQL: SELECT person, time, bsl,
       SUM(bsl) OVER win    AS bsl_sum,
       AVG(bsl) OVER win    AS bsl_avg,
       MIN(bsl) OVER win    AS bsl_min,
       MAX(bsl) over win as bsl_max,
       FIRST_VALUE(bsl) OVER win as bsl_start,
       LAST_VALUE(bsl) OVER win as bsl_end 
FROM df
WINDOW win AS (PARTITION BY person ORDER BY time)]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [54]:
g = df.groupby('person')
g.first()

,time,bsl
person,,
A,3,129
B,6,396
C,9,347
D,7,98


In [55]:
df_ = df.copy()
df_ = df_.sort_values(['person', 'time'])
df_['bsl_sum'] = df_.groupby(['person'])['bsl'].cumsum()
df_['bsl_avg'] = df_.groupby(['person']).expanding().agg({'bsl': 'mean'}).values
df_['bsl_min'] = df_.groupby(['person'])['bsl'].cummin()
df_['bsl_max'] = df_.groupby(['person'])['bsl'].cummax()
df_['bsl_start'] = df_.groupby(['person'])['bsl'].transform('first')
df_['bsl_end'] = df_.groupby(['person'])['bsl'].transform('last')
df_

,person,time,bsl,bsl_sum,bsl_avg,bsl_min,bsl_max,bsl_start,bsl_end
8,A,0,115,115,115.000000,115,115,115,129
5,A,2,237,352,176.000000,115,237,115,129
2,A,3,129,481,160.333333,115,237,115,129
7,B,5,86,86,86.000000,86,86,86,396
3,B,6,396,482,241.000000,86,396,86,396
4,C,1,107,107,107.000000,107,107,107,347
1,C,9,347,454,227.000000,107,347,107,347
6,D,5,89,89,89.000000,89,89,89,98
9,D,5,221,310,155.000000,89,221,89,98
0,D,7,98,408,136.000000,89,221,89,98


### Using rows and range to constrain windows

In [56]:
%%sql

SELECT person, time, bsl, 
       GROUP_CONCAT(CAST(bsl AS TEXT), ', ') OVER win AS vals,
       SUM(bsl) OVER win AS bsl_sum,
       AVG(bsl) OVER win AS bsl_avg
FROM df
WINDOW win AS (
    PARTITION BY person
    ORDER BY time
    ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING
)
ORDER BY person, time;

   sqlite://
 * sqlite:///
(sqlite3.OperationalError) no such table: df
[SQL: SELECT person, time, bsl, 
       GROUP_CONCAT(CAST(bsl AS TEXT), ', ') OVER win AS vals,
       SUM(bsl) OVER win AS bsl_sum,
       AVG(bsl) OVER win AS bsl_avg
FROM df
WINDOW win AS (
    PARTITION BY person
    ORDER BY time
    ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING
)
ORDER BY person, time;]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [57]:
df_ = df.copy()
df_ = df_.sort_values(['person', 'time'])
df_['bsl_sum'] = (df_.groupby('person').
                  bsl.apply(lambda x: x.rolling(3, min_periods=0, center=True).sum()))
df_['bsl_avg'] = (df_.groupby('person').
                  bsl.apply(lambda x: x.rolling(3, min_periods=0, center=True).mean()))
df_

,person,time,bsl,bsl_sum,bsl_avg
8,A,0,115,352.0,176.000000
5,A,2,237,481.0,160.333333
2,A,3,129,366.0,183.000000
7,B,5,86,482.0,241.000000
3,B,6,396,482.0,241.000000
4,C,1,107,454.0,227.000000
1,C,9,347,454.0,227.000000
6,D,5,89,310.0,155.000000
9,D,5,221,408.0,136.000000
0,D,7,98,319.0,159.500000


Revisit use of `LAST_VALUE` function

In [58]:
%%sql

SELECT person, time, bsl,
       SUM(bsl) OVER win    AS bsl_sum,
       AVG(bsl) OVER win    AS bsl_avg,
       MIN(bsl) OVER win    AS bsl_min,
       MAX(bsl) over win as bsl_max,
       FIRST_VALUE(bsl) OVER win as bsl_start,
       LAST_VALUE(bsl) OVER (
           PARTITION BY person ORDER BY time 
           ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING -- specify the current window
       ) as bsl_end 
FROM df
WINDOW win AS (PARTITION BY person ORDER BY time) 

   sqlite://
 * sqlite:///
(sqlite3.OperationalError) no such table: df
[SQL: SELECT person, time, bsl,
       SUM(bsl) OVER win    AS bsl_sum,
       AVG(bsl) OVER win    AS bsl_avg,
       MIN(bsl) OVER win    AS bsl_min,
       MAX(bsl) over win as bsl_max,
       FIRST_VALUE(bsl) OVER win as bsl_start,
       LAST_VALUE(bsl) OVER (
           PARTITION BY person ORDER BY time 
           ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING -- specify the current window
       ) as bsl_end 
FROM df
WINDOW win AS (PARTITION BY person ORDER BY time)]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


### Frames using Rows and Range

For Range, all rows with the same ORDER BY value are considered peers.

In [59]:
%%sql

SELECT person, time, bsl, 
       GROUP_CONCAT(CAST(bsl AS TEXT), ', ') OVER win AS vals,
       SUM(bsl) OVER win AS bsl_sum,
       AVG(bsl) OVER win AS bsl_avg
FROM df
WINDOW win AS (
    ORDER BY person, time
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
)
ORDER BY person, time;

   sqlite://
 * sqlite:///
(sqlite3.OperationalError) no such table: df
[SQL: SELECT person, time, bsl, 
       GROUP_CONCAT(CAST(bsl AS TEXT), ', ') OVER win AS vals,
       SUM(bsl) OVER win AS bsl_sum,
       AVG(bsl) OVER win AS bsl_avg
FROM df
WINDOW win AS (
    ORDER BY person, time
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
)
ORDER BY person, time;]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [60]:
df_ = df.copy()
df_ = df_.sort_values(['person', 'time'])
df_['bsl_sum'] = df_.bsl.expanding().sum().astype('int')
df_['bsl_avg'] = df_.bsl.expanding().mean()
df_

,person,time,bsl,bsl_sum,bsl_avg
8,A,0,115,115,115.000000
5,A,2,237,352,176.000000
2,A,3,129,481,160.333333
7,B,5,86,567,141.750000
3,B,6,396,963,192.600000
4,C,1,107,1070,178.333333
1,C,9,347,1417,202.428571
6,D,5,89,1506,188.250000
9,D,5,221,1727,191.888889
0,D,7,98,1825,182.500000


In [61]:
%%sql

SELECT person, time, bsl, 
       GROUP_CONCAT(CAST(bsl AS TEXT), ', ') OVER win AS vals,
       SUM(bsl) OVER win AS bsl_sum,
       AVG(bsl) OVER win AS bsl_avg
FROM df
WINDOW win AS (
    ORDER BY person
    RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
)
ORDER BY person, time;

   sqlite://
 * sqlite:///
(sqlite3.OperationalError) no such table: df
[SQL: SELECT person, time, bsl, 
       GROUP_CONCAT(CAST(bsl AS TEXT), ', ') OVER win AS vals,
       SUM(bsl) OVER win AS bsl_sum,
       AVG(bsl) OVER win AS bsl_avg
FROM df
WINDOW win AS (
    ORDER BY person
    RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
)
ORDER BY person, time;]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [62]:
df_ = df.copy()
df_ = df_.sort_values(['person', 'time'])
df_['bsl_sum'] =  df_['bsl'].cumsum().groupby(df_['person']).transform('last')
df_['bsl_sum'] =  df_['bsl'].expanding().mean().groupby(df_['person']).transform('last')
df_

,person,time,bsl,bsl_sum
8,A,0,115,160.333333
5,A,2,237,160.333333
2,A,3,129,160.333333
7,B,5,86,192.600000
3,B,6,396,192.600000
4,C,1,107,202.428571
1,C,9,347,202.428571
6,D,5,89,182.500000
9,D,5,221,182.500000
0,D,7,98,182.500000


#### Solution to recursive Fibonacci

In [ ]:
def fib_r(n):
    if n == 0:
        return 1
    elif n == 1:
        return 1
    else:
        return fib_r(n-2) + fib_r(n-1)